In [ ]:
import wikipedia
import requests
from pathlib import Path
from llama_index.core import SimpleDirectoryReader, Document

destinations = [
    "Paris", "Tokyo", "New York City", "Rome", "Sydney",
    "Barcelona", "Dubai", "Bali", "London", "Machu Picchu"
]

data_path = Path("travel_data")
data_path.mkdir(exist_ok=True)

def fetch_destination_data(destination):
    # 获取维基百科页面内容
    page = wikipedia.page(destination)
    text_content = page.content
  
    # 保存文本内容
    with open(data_path / f"{destination}.txt", "w", encoding="utf-8") as f:
        f.write(text_content)
  

    # 下载图片（这里只下载第一张图片作为示例）
    image_url = page.images[0]
    image_path = data_path / f"{destination}.jpg"
    response = requests.get(image_url)
    with open(image_path, "wb") as f:
        f.write(response.content)
  

    return Document(text=text_content, metadata={"image": str(image_path), "title": destination})

documents = [fetch_destination_data(dest) for dest in destinations]
print(f"Collected data for {len(documents)} destinations.")


In [ ]:
from llama_index.core import Settings
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.vector_stores import ChromaVectorStore
from llama_index.core.indices import MultiModalVectorStoreIndex
from llama_index.core.storage import StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding

# 设置OpenAI API密钥
import os
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"

# 配置全局设置
Settings.embed_model = OpenAIEmbedding()
Settings.node_parser = SimpleNodeParser.from_defaults(chunk_size=1024, chunk_overlap=20)

# 创建Chroma向量存储
vector_store = ChromaVectorStore(collection_name="travel_assistant")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# 构建多模态索引
index = MultiModalVectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

print("Multimodal index built successfully.")


In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.llms import OpenAI
from llama_index.core.prompts import PromptTemplate

# 创建自定义提示模板
custom_prompt = PromptTemplate(
    "You are a knowledgeable travel assistant. Based on the context provided, "
    "answer the user's question about travel destinations. If there's relevant "
    "image information, mention it in your response.\n"
    "Context: {context_str}\n"
    "Human: {query_str}\n"
    "Assistant: "
)

# 创建查询引擎
retriever = index.as_retriever(similarity_top_k=3)
llm = OpenAI(model="gpt-4")
query_engine = RetrieverQueryEngine.from_args(
    retriever,
    llm=llm,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
    text_qa_template=custom_prompt
)

print("Query engine created successfully.")


In [ ]:
import textwrap
from PIL import Image

def display_response(response):
    print("\nAssistant:")
    # 打印文本响应
    print(textwrap.fill(str(response), width=80))
  
    # 显示相关图片
    for node in response.source_nodes:
        if 'image' in node.node.metadata:
            image_path = node.node.metadata['image']
            print(f"\nRelevant image from {node.node.metadata['title']}:")
            Image.open(image_path).show()

def travel_assistant():
    print("Welcome to the Travel Assistant!")
    print("Ask me anything about the following destinations:")
    print(", ".join(destinations))
  

    while True:
        query = input("\nHuman: ")
        if query.lower() in ['exit', 'quit', 'bye']:
            print("Thank you for using the Travel Assistant. Goodbye!")
            break
      

        response = query_engine.query(query)
        display_response(response)

# 运行旅游助手
travel_assistant()


In [ ]:
def collect_feedback(query, response):
    print("\nWas this response helpful? (yes/no)")
    feedback = input().lower()
    if feedback == 'no':
        print("What could be improved?")
        improvement = input()
        # 在这里，你可以将反馈存储到数据库中，用于后续分析和改进
        print("Thank you for your feedback. We'll use it to improve our system.")
    else:
        print("Thank you for your feedback!")
    return feedback

# 修改travel_assistant函数，加入反馈收集
def travel_assistant():
    print("Welcome to the Travel Assistant!")
    print("Ask me anything about the following destinations:")
    print(", ".join(destinations))
  
    while True:
        query = input("\nHuman: ")
        if query.lower() in ['exit', 'quit', 'bye']:
            print("Thank you for using the Travel Assistant. Goodbye!")
            break
      

        response = query_engine.query(query)
        display_response(response)
        feedback = collect_feedback(query, response)
      

        # 这里可以添加逻辑来基于反馈调整系统，例如更新索引或微调模型

# 运行改进后的旅游助手
travel_assistant()

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/query', methods=['POST'])
def query():
    data = request.json
    query_text = data['query']
    response = query_engine.query(query_text)
  

    # 这里需要修改display_response函数以返回JSON格式的数据
    result = display_response(response)
  

    return jsonify(result)

if __name__ == '__main__':
    app.run(debug=True)
